In [3]:
using Distributions

In [4]:
type Individual
    m::Int64
    p::Int64
    v_e::Float64
    mu::Float64
    mu_b::Float64
    mu_sigma::Float64
    y::Array{Float64, 1}
    b::Array{Int64, 2}
    x::Array{Float64, 1}
    z::Array{Float64, 1}
    
    Individual(m, p, v_e, mu, mu_b, mu_sigma) = new(2*m, p, v_e, mu, mu_b, mu_sigma)
end  

type Population
    n::Int64
    base_ind::Individual
    generation::Int64
    fitness::Array{Float64, 1}
    theta::Array{Float64, 1}
    omega::Array{Float64, 2}
    pop::Array{Individual, 1}
    Population(n, base_ind, generation) = new(n, base_ind, generation)
end

In [8]:
function Population(n::Int, ind::Individual, pop_array::Array{Individual, 1})
    new_pop = Population(n, ind, 0)
    new_pop.generation = 0
    new_pop.pop = pop_array
    new_pop
end

function RandomInd!(ind::Individual, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    ind.y = rand(Normal(y_mean, g_sigma), ind.m)
    ind.b = rand(Bernoulli(), ind.p, ind.m)
    ind.x = ind.b * ind.y
    ind.z = ind.x + rand(Normal(0, ind.v_e), ind.p)
end

function RandomInd(ind::Individual, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    new_ind = Individual(ind.m/2, ind.p, ind.v_e, ind.mu, ind.mu_b, ind.mu_sigma)
    RandomInd!(new_ind, y_mean, g_sigma)
    new_ind
end

function RandomPop!(pop::Population, y_mean::Float64 = 0.0, g_sigma::Float64 = ind.mu_sigma)
    pop.pop = Array(Individual, pop.n)
    pop.generation = 0
    for i = 1:pop.n
        pop.pop[i] = RandomInd(pop.base_ind, y_mean, g_sigma)
    end
end


import Base.getindex
import Base.copy

function getindex(pop::Population, i::Integer)
    getindex(pop.pop, i)
end

function getindex(pop::Population, s::UnitRange)
    Population(length(s), pop.base_ind, getindex(pop.pop, s))
end

function append!(pop::Population, ind::Individual)
    pop.pop = [pop.pop; ind]
    pop.n = length(pop.pop)
end

function append!(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    pop1.pop = [pop1.pop; pop2.pop]
    pop.n = length(pop1.pop)
end

function join(pop1::Population, pop2::Population)
    # COMPARE BASE IND!
    new_pop = Population(pop1.n + pop2.n, pop1.base_ind)
    new_pop.pop = [pop1.pop; pop2.pop]
    new_pop
end

function copy!(source::Population, sink::Population)
    sink.n = source.n
    sink.base_ind = source.base_ind
    sink.generation = source.generation
    sink.fitness = source.fitness
    sink.theta = source.theta
    sink.omega = source.omega
    sink.pop = source.pop
end

function copy(source::Population)
    new_pop = Population(source.n, source.base_ind, source.generation)
    copy!(source, new_pop)
    new_pop
end
    
function mutation!(ind::Individual)
    mutation_y = rand(Binomial(ind.m, ind.mu))
    if(mutation_y > 0)
        for k = range(1, mutation_y)
            i = rand(DiscreteUniform(1, ind.m))
            ind.y[i] += rand(Normal(0, ind.mu_sigma))
        end
    end
    mutation_b = rand(Binomial(ind.m * ind.p, ind.mu_b))
    if(mutation_b > 0)
        for k = range(1, mutation_b)
            i = rand(DiscreteUniform(1, ind.p))
            j = rand(DiscreteUniform(1, ind.m))
            ind.b[i,j] = ind.b[i, j] == 1 ? 0 : 1
        end
    end
end

function mutation!(pop::Population)
    for k = 1:pop.n
        mutation!(pop.pop[k])
    end
end

function fitness(ind::Individual, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    ind_fitness = pdf(MvNormal(theta, omega), ind.z)
    isnan(ind_fitness) ? 0 : ind_fitness
end

function fitness(pop::Population, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    fitness_array = [fitness(pop.pop[k], theta, omega) for k = 1:pop.n]
    total_fitness = sum(fitness_array)
    if(total_fitness < 0.000000001)
        fitness_array = ones(Float64, pop.n)
    end
    fitness_array
end

function fitness(pop::Population)
    fitness_array = [fitness(pop.pop[k], pop.theta, pop.omega) for k = 1:pop.n]
    total_fitness = sum(fitness_array)
    if(total_fitness < 0.000000001)
        fitness_array = ones(Float64, pop.n)
    end
    fitness_array
end

function fitness!(pop::Population, theta::Array{Float64, 1}, omega::Array{Float64, 2})
    pop.fitness = fitness(pop, theta, omega)
end

function fitness!(pop::Population)
    pop.fitness = fitness(pop)
end

function cross(ind_1::Individual, ind_2::Individual, new_ind::Individual)
    alele_1 = rand(DiscreteUniform(0, 1), Int(ind_1.m/2))
    alele_2 = rand(DiscreteUniform(0, 1), Int(ind_2.m/2))
    for locus = range(2, 2, 5)
        new_ind.y[   locus - 1] = ind_1.y[   (locus - 1) + alele_1[convert(Int64, locus/2)]]
        new_ind.y[   locus]     = ind_2.y[   (locus - 1) + alele_2[convert(Int64, locus/2)]]
        new_ind.b[:, locus - 1] = ind_1.b[:, (locus - 1) + alele_1[convert(Int64, locus/2)]]
        new_ind.b[:, locus]     = ind_2.b[:, (locus - 1) + alele_2[convert(Int64, locus/2)]]
    end
    new_ind.x = new_ind.b * new_ind.y
    new_ind.z = new_ind.x + rand(Normal(0, new_ind.v_e), new_ind.p)
end

function cross(ind_1::Individual, ind_2::Individual)
    new_ind = Individual(ind_1.m/2, ind_1.p, ind_1.v_e, ind_1.mu, ind_1.mu_b, ind_1.mu_sigma)
    new_ind.y = zeros(Float64, new_ind.m)
    new_ind.b = zeros(Float64, new_ind.p, new_ind.m)
    cross(ind_1, ind_2, new_ind)
    new_ind
end   

function choose_mates(pop::Population)
    matings = rand(Multinomial(pop.n, pop.fitness/sum(pop.fitness)), 1)
    mates = zeros(Float64, pop.n)
    l = 1
    for k = 1:pop.n
        if(matings[k] > 0)
            for(i = 1:matings[k])
                mates[l] = k
                l += 1
            end
        end
    end
    round(Int64, shuffle!(mates))
end

function next_generation(pop::Population; selective = true)
    holder_pop = copy(pop)
    next_generation!(pop, holder_pop, selective)
end

function next_generation!(pop::Population, holder_pop::Population; selective = true)
    if (selective)
        fitness!(pop)
    else
        pop.fitness = ones(Float64, pop.n)
    end
    mutation!(pop)
    sires = choose_mates(pop)
    dames = choose_mates(pop)
    for i in 1:pop.n
        #if(i % 10 == 0) 
        #    println("cross n:", i)
        #end
        cross(pop[sires[i]], pop[dames[i]], holder_pop.pop[i])
    end
    holder_pop.generation = pop.generation + 1
    copy!(holder_pop, pop)
end 

function moments(pop::Population)
    ys = convert(Array{Float64, 2}, reshape([ind.y[i] for ind in pop.pop, i in 1:pop.base_ind.m], pop.n, pop.base_ind.m))
    xs = convert(Array{Float64, 2}, reshape([ind.x[i] for ind in pop.pop, i in 1:pop.base_ind.p], pop.n, pop.base_ind.p))
    zs = convert(Array{Float64, 2}, reshape([ind.z[i] for ind in pop.pop, i in 1:pop.base_ind.p], pop.n, pop.base_ind.p))
    mean_b = zeros(Float64, pop.base_ind.p, pop.base_ind.m)
    for i in 1:pop.n
        mean_b += pop[i].b
    end
    mean_y = squeeze(mean(ys, 1), 1)
    mean_x = squeeze(mean(xs, 1), 1)
    mean_z = squeeze(mean(zs, 1), 1)
    mean_b /= pop.n

    P = cov(zs)
    G = cov(xs)
    h2 = diag(G)./diag(P)
    corrG = cor(xs)
    corrP = cor(zs)

    Dict([("mean_y", mean_y), 
          ("mean_b", mean_b), 
          ("mean_x", mean_x),
          ("mean_z", mean_z), 
          ("P", P), 
          ("G", G), 
          ("h2", h2), 
          ("corrG", corrG),
          ("corrP", corrP)])
end

moments (generic function with 1 method)

In [9]:
function run_pop(gen_drift, gen_stab)
    global ind = Individual(500, 20, 0.8, 0.005, 0.001, 0.2)
    pop = Population(5000, ind, 0)
    RandomPop!(pop)
    pop.theta = zeros(Float64, ind.p)
    pop.omega = diagm(ones(Float64, ind.p))
    fitness!(pop)
    holder_pop = copy(pop)
    for i = 1:gen_drift
        if(i % 10 == 0) 
            println("Generation :", i)
        end
        next_generation!(pop, holder_pop, selective = false)
    end
    for i = 1:gen_stab
        if(i % 10 == 0) 
            println("Selective Generation :", i)
        end
        next_generation!(pop, holder_pop, selective = true)
    end
end

run_pop (generic function with 1 method)

In [12]:
@time run_pop(100, 100)

Generation :10
Generation :20
Generation :30
Generation :40
Generation :50
Generation :60
Generation :70
Generation :80
Generation :90
Generation :100
Selective Generation :10
Selective Generation :20
Selective Generation :30
Selective Generation :40
Selective Generation :50
Selective Generation :60
Selective Generation :70
Selective Generation :80
Selective Generation :90
Selective Generation :100
 69.141343 seconds (39.14 M allocations: 13.575 GB, 3.23% gc time)
